# 02: DGAN

The implementation of the DGAN comes from the [Gretel-synthetics-Repository](https://github.com/gretelai/gretel-synthetics)

In [ ]:
import os

import numpy as np
import pandas as pd

from synthesizers.dgan.dgan import DGAN
from synthesizers.dgan.config import DGANConfig, OutputType


In [ ]:
windows = np.load('data/wesad/wesad_windows.npy')
labels = np.load('data/wesad/wesad_labels.npy')

In [ ]:
config = DGANConfig(
    max_sequence_len=windows.shape[1],
    sample_len=60,
    batch_size=min(1000, windows.shape[0]),
    apply_feature_scaling=False,
    apply_example_scaling=False,
    use_attribute_discriminator=False,
    generator_learning_rate=1e-4,
    discriminator_learning_rate=1e-4,
    epochs=10000,
    cuda=True,
    #mps=True
)

print(config.batch_size)

# Train DGAN model
model = DGAN(config)

In [ ]:
model.train_numpy(
    windows,
    feature_types=[OutputType.CONTINUOUS] * windows.shape[2]
)
os.system("say 'DGAN IST FERTIG'")

In [ ]:
model.save("models/dgan/generator")

In [ ]:
model.load("models/dgan/generator")

In [ ]:
num_syn_subjects_list = [1] + [i for i in range(10, 101, 10)] + [i for i in range(200, 1001, 100)] 

# Generate synthetic subject
for num_syn_subjects in num_syn_subjects_list:
    num_syn_samples = windows.shape[0] // 16
    _, gen_data = model.generate_numpy(num_syn_samples)
    gen_data[:, :, 6] = np.where(gen_data[:, :, 6] >= 0.5, 1, 0)
    with open (f"data/syn/dgan/no_dp/aug/syn_subject_{num_syn_subjects}.npy", "wb") as f:
       np.save(f, gen_data)

In [ ]:
# Generate synthetic dataset
num_syn_samples = windows.shape[0]
_, gen_data = model.generate_numpy(num_syn_samples)
gen_data[:, :, 6] = np.where(gen_data[:, :, 6] >= 0.5, 1, 0)
with open (f"data/syn/dgan/no_dp/syn_dataset_{num_syn_samples}_10_subjs.npy", "wb") as f:
   np.save(f, gen_data)